In [1]:
from dashboard.db import DataFetcher, DBCreds

db_creds = DBCreds(
    host="localhost",
    port="3306",
    user="local",
    password="local",
    database="jobs_data"
)
db_data = DataFetcher(db_creds)

In [4]:
from datetime import datetime 

start_date = datetime(2021, 1, 1)
end_date = datetime(2025, 1, 31)

data = db_data.fetch_data_bw_dates(start_date, end_date)

In [24]:
sdf

,job_function,counts
0,Engineering and Information Technology,1839
1,Information Technology,1101
2,Other,349
3,Engineering,310
4,Information Technology and Engineering,243
...,...,...
402,"Accounting/Auditing, Analyst, and Finance",1
403,Accounting/Auditing and Analyst,1
404,Sales and Marketing,1
405,Sales and Management,1


In [26]:
sdf = df.groupby(['job_function']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)

In [27]:
sdf

,job_function,counts,x
0,Engineering and Information Technology,1839,0
1,Information Technology,1101,1
2,Other,349,2
3,Engineering,310,3
4,Information Technology and Engineering,243,4
...,...,...,...
402,"Accounting/Auditing, Analyst, and Finance",1,402
403,Accounting/Auditing and Analyst,1,403
404,Sales and Marketing,1,404
405,Sales and Management,1,405


In [ ]:
# write a function which would assign x and y values to each job function to be plotted on a bubble chart without overlapping
# the bubbles
# the function should take the dataframe and the column names as input and return a dataframe with x and y values assigned
# to each job function
def assign_x_y(df, size_col):
    df = df.sort_values(by=size_col, ascending=False).reset_index(drop=True)
    df["x"] = df.index
    df["y"] = 0
    return df

In [42]:
import plotly.express as px
import pandas as pd

df = data
sdf = df.groupby(['job_function']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
# draw a tree map in plotly
fig = px.treemap(sdf, path=["job_function"], values="counts", title="Job Functions")
# save the figure
fig.write_html("job_functions.html")

In [38]:
!pip install nbformat>=4.2.0


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
fig.show()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [100]:
from dashboard.db import DataFetcher, DBCreds
from dashboard.plots.jobs_plotter import JobsPlotter
from dashboard.plots.skills_plotter import SkillsPlotter



# MySQL datafetcher
db_creds = DBCreds(
    host="localhost",
    port="3306",
    user="local",
    password="local",
    database="jobs_data"
)
db_data = DataFetcher(db_creds)

In [101]:
from datetime import datetime 

start_date = datetime(2021, 1, 1)
end_date = datetime(2025, 1, 31)

skills_data = db_data.fetch_skills_bw_dates(start_date, end_date)
clusters_data = db_data.fetch_skill_clusters()

In [97]:
ranking_weightage = 0.2

skills_data['skill_weightage'] = 1 - (skills_data['skill_rank'] - 1) * ranking_weightage / 10
cluster_name_map = dict(zip(clusters_data['cluster_id'], clusters_data['cluster_head_tag']))
skills_data['cluster_name'] = skills_data['cluster_id'].apply(lambda x: cluster_name_map[int(x)])
gdf = skills_data.groupby(['cluster_name']).agg({'skill_weightage': 'sum'}).reset_index()
#gdf = skills_data.groupby(['cluster_id']).agg({'skill_weightage': 'sum', 'cluster_name': 'max'}).reset_index()
df = gdf
cluster_df = clusters_data

In [98]:
import json
def json_to_commasep( x):
        lis = json.loads(x)
        return ', '.join(lis)
    
cluster_df['cluster_tags'] = cluster_df['cluster_tags'].apply(json_to_commasep)
cluster_df.rename(columns={'cluster_head_tag': 'cluster_name'}, inplace=True)
#cluster_df['cluster_name'] = cluster_df['cluster_name'].apply(lambda x: str(x))
joined_df = df.merge(cluster_df, left_on='cluster_name', right_on='cluster_name', how='left')[['cluster_name', 'cluster_tags', 'skill_weightage']].sort_values(by='skill_weightage', ascending=False).drop_duplicates(subset='cluster_name').reset_index(drop=True)

In [99]:
joined_df

,cluster_name,cluster_tags,skill_weightage
0,Machine Learning,Machine Learning,1802.68
1,System Design Architecture Model Experience (S...,"design, architecture, Model, system, experience",1251.94
2,Database Querying,SQL,1249.12
3,Data Visualization Tools,"Power BI, Tableau, C, Spark, MLOps",1198.20
4,Human Experience,"experience, skill, knowledge, Technical",1126.90
5,Data Structures and Data Analysis,"Data, Data structure",1080.00
6,Communication Skills,"communication skill, skills Communication",1033.74
7,System Development and Management,"detail, Attention, management, system, develop...",966.16
8,Problem Solving Skills,"problem solving, solving skills",950.74
9,Team Development,"Leadership, Project management, Collaboration,...",946.96


In [108]:
cluster_id=3
skills = ", ".join(skills_data[skills_data['cluster_id'] == str(cluster_id)]['skill'].values.tolist())

In [111]:
len(skills)

34134